In [4]:
import pandas as pd
import numpy as np
from pathlib import Path

**Import Data**

In [5]:
base = Path("/content/sample_data/data")
files = list(base.glob("*.csv"))

In [6]:
dfs = {}
for f in files:
    df = pd.read_csv(f)
    dfs[f.name] = df
    print(f"Loaded {f.name} — {df.shape[0]} rows, {df.shape[1]} columns")

Loaded Stores.csv — 7 rows, 4 columns
Loaded Products.csv — 17 rows, 4 columns
Loaded Promotions.csv — 12 rows, 5 columns
Loaded Customers.csv — 12 rows, 5 columns
Loaded BundledProductsPromotions.csv — 9 rows, 2 columns
Loaded Sales.csv — 135 rows, 6 columns
Loaded BundledProducts.csv — 9 rows, 2 columns
Loaded BundledProducts_Products.csv — 20 rows, 2 columns
Loaded Categories.csv — 5 rows, 2 columns


**Check if data loaded correctly**

In [7]:
dfs["Sales.csv"].head()

,OrderID,CustomerID,BundledProductID,OrderDate,SalesAmount,StoreID
0,1001.0,5001.0,3001.0,15/01/2024,150.0,101.0
1,1002.0,5002.0,3002.0,16/01/2024,200.0,102.0
2,1003.0,5003.0,3003.0,17/01/2024,300.0,101.0
3,1004.0,5001.0,3001.0,15/02/2024,100.0,103.0
4,1005.0,5002.0,3002.0,16/02/2024,250.0,101.0


In [8]:
for name, df in dfs.items():
    print(name)
    display(df.head())

Stores.csv


,StoreID,StoreName,Location,Manager
0,101,Store A,New York,Alice
1,102,Store B,Los Angeles,Bob
2,103,Store C,Chicago,Charlie
3,104,Store D,San Francisco,David
4,105,Store E,Miami,Eve


Products.csv


,ProductID,ProductName,CategoryID,Price
0,2001,Product A,100,150.0
1,2002,Product B,101,200.0
2,2003,Product C,102,300.0
3,2004,Product D,103,100.0
4,2005,Product E,104,350.0


Promotions.csv


...

,BundledProductID,BundledProductName
0,3001,Bundle A
1,3002,Bundle B
2,3003,Bundle C
3,3004,Bundle D
4,3005,Bundle E


BundledProducts_Products.csv


,BundledProductID,ProductID
0,3001,2001
1,3001,2002
2,3001,2003
3,3002,2004
4,3002,2005


Categories.csv


,CategoryID,CategoryName
0,100,Electronics
1,101,Clothing
2,102,Home & Kitchen
3,103,Sports
4,104,Books


**Data Profilling Summary**

In [9]:

summary = []

for name, df in dfs.items():
    # basic info
    total_rows = len(df)
    total_cols = len(df.columns)
    total_cells = total_rows * total_cols

    # missing values
    missing_cells = df.isna().sum().sum()
    missing_pct_cells = round((missing_cells / total_cells) * 100, 2)
    missing_pct_rows = round(df.isna().any(axis=1).mean() * 100, 2)

    # duplicates
    dup_count = df.duplicated().sum()
    dup_pct = round((dup_count / total_rows) * 100, 2) if total_rows > 0 else 0

    summary.append({
        "Dataset": name,
        "Rows": total_rows,
        "Columns": total_cols,
        "Missing Cells %": missing_pct_cells,
        "Rows w/ Missing %": missing_pct_rows,
        "Duplicates": dup_count,
        "Duplicates %": dup_pct
    })

summary_df = pd.DataFrame(summary)
summary_df


,Dataset,Rows,Columns,Missing Cells %,Rows w/ Missing %,Duplicates,Duplicates %
0,Stores.csv,7,4,0.00,0.00,0,0.00
1,Products.csv,17,4,1.47,5.88,1,5.88
2,Promotions.csv,12,5,0.00,0.00,0,0.00
3,Customers.csv,12,5,3.33,16.67,1,8.33
4,BundledProductsPromotions.csv,9,2,0.00,0.00,0,0.00
5,Sales.csv,135,6,3.70,11.85,5,3.70
6,BundledProducts.csv,9,2,0.00,0.00,0,0.00
7,BundledProducts_Products.csv,20,2,0.00,0.00,0,0.00
8,Categories.csv,5,2,0.00,0.00,0,0.00


  Obseravation:
  - Duplicates need to be deduplicated
  - Missing values for [Sales], [Products] and [Customers] need to be handled, considering the missing percentage is relatively small, we can either safely drop some rows, or handle numeric with median, category with 'unknown'etc. however if the primary key is missing we need to handle them specifically. If foreign keys are missing we need to investigate and think about an approach
  - For outliers we can have better visualization in PBI later on

**Clean the data**

In [10]:
cleaned_dfs = {}

In [11]:
for name, df in dfs.items():
    #Trim whitespaces
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    #Standardise the nulls before removing null values
    df = df.replace(["", " ", "None", "NULL", "null", "N/A", "na", "-", "--"], np.nan)

    # Deduplicate
    df = df.drop_duplicates()

    #If all columns are missing for one row, safe to drop
    all_null_before = df.isna().all(axis=1).sum()
    df = df.dropna(how="all")

    cleaned_dfs[name] = df



/tmp/ipython-input-2832348431.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/tmp/ipython-input-2832348431.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/tmp/ipython-input-2832348431.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


**Define and validate primary keys**

For table [BundledProducts_Products], bundledproductid and productid is many to many relationship, we can create a surrogate key for this table

For table [BundledProductsPromotions], the relationship between bundledproductid and promotionid looks like 1:1 however from business perspective many to many relationship makes more sense, we will also create a surrogate key for this table

In [18]:

bp = cleaned_dfs["BundledProducts_Products.csv"].copy()

# keep only unique pairs, then assign a bridge PK
# Used the auto-increment integers as surrogate key
pair_cols = ["BundledProductID", "ProductID"]
bp = bp.drop_duplicates(subset=pair_cols).reset_index(drop=True)
bp.insert(0, "BP_Prod_BridgeID", range(1, len(bp) + 1))

cleaned_dfs["BundledProducts_Products.csv"] = bp

# BundledProductsPromotions bridge
bpp = cleaned_dfs["BundledProductsPromotions.csv"].copy()

pair_cols2 = ["BundledProductID", "PromotionID"]
bpp = bpp.drop_duplicates(subset=pair_cols2).reset_index(drop=True)
bpp.insert(0, "BP_Promo_BridgeID", range(1, len(bpp) + 1))

cleaned_dfs["BundledProductsPromotions.csv"] = bpp


In [23]:
primary_keys = {
    "Products.csv": ["ProductID"],
    "BundledProducts.csv": ["BundledProductID"],
    "Promotions.csv": ["PromotionID"],
    "Customers.csv": ["CustomerID"],
    "Stores.csv": ["StoreID"],
    "Categories.csv": ["CategoryID"],
    "BundledProducts_Products.csv": ["BP_Prod_BridgeID"],
    "BundledProductsPromotions.csv": ["BP_Promo_BridgeID"],
    "Sales.csv": ["OrderID"]
}


pk_results = []
for name, df in cleaned_dfs.items():
    pks = primary_keys.get(name, [])
    if not pks:
        pk_results.append({"Dataset": name, "PK": "None defined", "Missing PK values": "N/A", "Duplicate PKs": "N/A"})
        continue

    missing_cols = [c for c in pks if c not in df.columns]
    if missing_cols:
        pk_results.append({"Dataset": name, "PK": f"Missing columns: {missing_cols}", "Missing PK values": "N/A", "Duplicate PKs": "N/A"})
        continue

    miss_rows = int(df[pks].isna().any(axis=1).sum())
    dup_rows  = int(df.duplicated(subset=pks).sum())
    pk_results.append({"Dataset": name, "PK": ", ".join(pks), "Missing PK values": miss_rows, "Duplicate PKs": dup_rows})

pd.DataFrame(pk_results)


,Dataset,PK,Missing PK values,Duplicate PKs
0,Stores.csv,StoreID,0,0
1,Products.csv,ProductID,0,0
2,Promotions.csv,PromotionID,0,0
3,Customers.csv,CustomerID,0,0
4,BundledProductsPromotions.csv,BP_Promo_BridgeID,0,0
5,Sales.csv,OrderID,0,0
6,BundledProducts.csv,BundledProductID,0,0
7,BundledProducts_Products.csv,BP_Prod_BridgeID,0,0
8,Categories.csv,CategoryID,0,0


Primary keys are all clean, then we need to check the foreign keys and missing values. By observation on the dataset, [Sales] table is the only one we need to handle specifically

In [24]:
sales = cleaned_dfs["Sales.csv"]

# Foreign keys in Sales
key_columns = [ "CustomerID", "StoreID", "BundledProductID"]

null_summary = []

for col in key_columns:
    null_count = int(sales[col].isna().sum())
    total = len(sales)
    pct = round((null_count / total) * 100, 2)
    null_summary.append({
        "Column": col,
        "Missing Count": null_count,
        "Missing %": pct
    })

missing_df = pd.DataFrame(null_summary)
display(missing_df)


,Column,Missing Count,Missing %
0,CustomerID,0,0.0
1,StoreID,0,0.0
2,BundledProductID,0,0.0


CustomerID in [Sales] have 8 null values, we can assign these as CustomerID = -1 and add CustomerID = -1 in [Customers] dimention table and capture it as 'Unknow' category, so we have a clear explanation on this and PBI will still join successfully on these rows

In [25]:
# Replace missing CustomerID in Sales with -1
sales["CustomerID"] = sales["CustomerID"].fillna(-1)

In [26]:
customers = cleaned_dfs["Customers.csv"]
if -1 not in customers["CustomerID"].values:
    unknown_row = {
        "CustomerID": -1,
        "CustomerName": "Unknown Customer"
    }
    customers = pd.concat([customers, pd.DataFrame([unknown_row])], ignore_index=True)

cleaned_dfs["Sales.csv"] = sales
cleaned_dfs["Customers.csv"] = customers

In [27]:
#Copy and run the data profilling on cleaned_dfs again
summary = []

for name, df in cleaned_dfs.items():
    # basic info
    total_rows = len(df)
    total_cols = len(df.columns)
    total_cells = total_rows * total_cols

    # missing values
    missing_cells = df.isna().sum().sum()
    missing_pct_cells = round((missing_cells / total_cells) * 100, 2)
    missing_pct_rows = round(df.isna().any(axis=1).mean() * 100, 2)

    # duplicates
    dup_count = df.duplicated().sum()
    dup_pct = round((dup_count / total_rows) * 100, 2) if total_rows > 0 else 0

    summary.append({
        "Dataset": name,
        "Rows": total_rows,
        "Columns": total_cols,
        "Missing Cells %": missing_pct_cells,
        "Rows w/ Missing %": missing_pct_rows,
        "Duplicates": dup_count,
        "Duplicates %": dup_pct
    })

summary_df = pd.DataFrame(summary)
summary_df

,Dataset,Rows,Columns,Missing Cells %,Rows w/ Missing %,Duplicates,Duplicates %
0,Stores.csv,7,4,0.00,0.00,0,0.0
1,Products.csv,16,4,1.56,6.25,0,0.0
2,Promotions.csv,12,5,0.00,0.00,0,0.0
3,Customers.csv,12,6,23.61,100.00,0,0.0
4,BundledProductsPromotions.csv,9,3,0.00,0.00,0,0.0
5,Sales.csv,129,6,1.29,7.75,0,0.0
6,BundledProducts.csv,9,2,0.00,0.00,0,0.0
7,BundledProducts_Products.csv,20,3,0.00,0.00,0,0.0
8,Categories.csv,5,2,0.00,0.00,0,0.0


Output Data

In [28]:
import os
import zipfile
from google.colab import files

output_folder = "cleaned_data"
os.makedirs(output_folder, exist_ok=True)

for name, df in cleaned_dfs.items():
    file_path = os.path.join(output_folder, name)
    df.to_csv(file_path, index=False)
    print(f"Saved: {file_path}")

zip_filename = "cleaned_data.zip"
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files_in_dir in os.walk(output_folder):
        for file in files_in_dir:
            zipf.write(os.path.join(root, file), arcname=file)

files.download(zip_filename)

Saved: cleaned_data/Stores.csv
Saved: cleaned_data/Products.csv
Saved: cleaned_data/Promotions.csv
Saved: cleaned_data/Customers.csv
Saved: cleaned_data/BundledProductsPromotions.csv
Saved: cleaned_data/Sales.csv
Saved: cleaned_data/BundledProducts.csv
Saved: cleaned_data/BundledProducts_Products.csv
Saved: cleaned_data/Categories.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>